In [ ]:
import SimpleITK as sitk
import nibabel as nib
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

# Seperate dsc / dce

In [ ]:
## Perfusion 이미지가 저장된 공간
srcDir  = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/original_data/target_img'

## 분리한 3차원 이미지를 저장할 공간
destDir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/target_img_seperate'
os.makedirs(destDir, exist_ok = True)

In [ ]:
total = 0
for (path, dirs, files) in os.walk(srcDir):
    if files:
        total += 1
        pt = os.path.basename(path)
        print("# {} : seperate img in {}".format(total, pt))
        
        
        
        for file in files:
            seq = file.split('.')[0].split('_')[-1]
            print("- {} is sperating now....".format(seq))
            
            file_path = os.path.join(path, file)
            
            # 이미지 불러오기
            img = sitk.ReadImage(file_path)
            img_data = sitk.GetArrayFromImage(img)
            print(img.GetSize())
            
            print(img.GetSpacing())
            print(img.GetOrigin())
            print(np.array(img.GetDirection()).reshape(4,4))
            
            # set 3D info - 4차원 정보들 중 3차원 정보에 해당하는 부분만 추출하여 분리된 3차원 이미지에 입혀주기
            origin_spacing = img.GetSpacing()[:3]
            origin_origin = img.GetOrigin()[:3]
            origin_direction = np.array(img.GetDirection()).reshape(4,4)[:3, :3].ravel(order = 'C')
            
            print(origin_spacing)
            print(origin_origin)
            print(origin_direction.reshape(3,3))
            
            # make path
            seq_dir = os.path.join(destDir, pt, seq)
            os.makedirs(seq_dir, exist_ok = True)
            
            # 하나의 Perfusion 영상에서 시간축으로 이어붙여진 3차원 영상을 하나씩 분리하는 과정
            check = 0
            for slice  in img_data:
                check += 1
                new_file_name = '{}_{}_{}.nii.gz'.format(check, pt, seq)
                final_path = os.path.join(seq_dir, new_file_name)
                print(check, slice.shape, new_file_name)
                
                
                if os.path.exists(final_path):
                    print("File already exists!!")
                    continue
                               
                slice_img = sitk.GetImageFromArray(slice)
                
                slice_img.SetOrigin(origin_origin)
                slice_img.SetDirection(origin_direction)
                slice_img.SetSpacing(origin_spacing)
                
                sitk.WriteImage(slice_img , final_path)
                          
            
            print()
        print()

# Co-registration each img

In [ ]:
## 분리된 Perfusion 영상들이 저장되어 있는 곳
srcDir  = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/target_img_seperate'

## Co-Registartion 한 Perfusion 영상을 저장할 곳
destDir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/target_img_seperate_registration'
os.makedirs(destDir, exist_ok = True)

## Co-Registration을 하기 위해 Reference로 삼을 영상이 저장된 곳(MASK를 얻기 위해 사용한 영상)
CT1_Dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/preprocessing_data/CT1_resample'

In [ ]:
pt_lst = os.listdir(srcDir)
pt_lst = sorted([int(pt) for pt in pt_lst])
pt_lst = [str(pt) for pt in pt_lst]
pt_lst

In [ ]:
## ANTS-Coregistartion을 이용하였습니다.

total = 0
for (path, dirs, files) in os.walk(srcDir):
    if files:
        total += 1
        seq = os.path.basename(path)
        pt = path.split(os.path.sep)[-2] 
        print("# {} : seperate img in pt: {} / seq : {}".format(total, pt, seq))
        
        # Reference 영상을 불러오기
        CT1_path = glob(os.path.join(CT1_Dir, '*{}*'.format(pt)))
        
        if len(CT1_path) != 1:
            print("Something wrong in CT1!!")
            print()
            continue
        
        print(CT1_path)
        CT1_path = CT1_path[0]
        
        
        seq_dir = os.path.join(destDir, pt, seq)
        os.makedirs(seq_dir, exist_ok = True)
        
        # 분리된 Perfusion 영상을 Reference에 각각 Co-Registration 하기
        check = 0
        for file in sorted(files):
            check += 1
            file_name = file.split('.')[0]
            in_path = os.path.join(path, file)
            
            new_file_name = '{}_registered.nii.gz'.format(file_name)
            print(check, new_file_name)
            
            outPath = os.path.join(seq_dir, new_file_name)
            
            if os.path.exists(outPath):
                print("File already exists!!")
                continue
                
                        
            !antsRegistration -d 3 -r [{CT1_path}, {in_path}, 1] \
                      -m mattes[{CT1_path}, {in_path}, 1, 32, regular, 0.1 ] \
                      -t translation[0.1] -c [100x50x25,1.e-8,20] -s 4x2x1vox -f 6x4x2 -l 1 \
                      -m mattes[{CT1_path}, {in_path} , 1 , 32, regular, 0.1] \
                      -t rigid[0.1] -c [100x50x25,1.e-8,20]  -s 4x2x1vox  -f 3x2x1 -l 1 -o [reg, {outPath}] -v 0
        
        
        
        print()